# Most attractive Apps for free

This project aims at analyse the current market of free apps and understand what kind of apps are more attractive to users.

We're gonna see what genre of apps our Company should develop


Let's read and explore the datasets

In [1]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [2]:
Apple_open=open('AppleStore.csv')
from csv import reader
Apple_read=reader(Apple_open)
Apple_data=list(Apple_read)
Apple_header= Apple_data[0]
Apple_data= Apple_data[1:]

In [3]:
Google_open=open('googleplaystore.csv')
from csv import reader
Google_read=reader(Google_open)
Google_data=list(Google_read)
Google_header=Google_data[0]
Google_data=Google_data[1:]

In [4]:
print(Apple_header, '\n')
explore_data(Apple_data,0,3,True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


Useful columns:

- 1 Track_name
- 4 Price
- 5 Rating count tot
- 6 Rating count ver
- 11 Prime genre


In [5]:
print(Google_header, '\n')
explore_data(Google_data,0,2,True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


Useful column:
- 0 App
- 1 Category
- 3 Reviews
- 5 Installs
- 6 Type
- 7 Price
- 9 Genre


According to this discussion https://www.kaggle.com/lava18/google-play-store-apps/discussion/164101

10472 row has wrong data. 'Category' Column is missing

In [6]:
print(Google_header, '\n', 'Columns: ', len(Google_header))
print(Google_data[10472], '\n', 'Columns: ', len(Google_data[10472]))

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 
 Columns:  13
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up'] 
 Columns:  12


This column will be removed

In [7]:
del Google_data[10472] #only use this once

## Removing duplicate apps

Let's create two lists for duplicate and unique apps:

In [8]:
duplicate_apps=[]
unique_apps=[]

for app in Google_data:
    name=app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

Now we count the number of duplicate apps and extract a sample of them:

In [9]:
print('Number of duplicate apps: ', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps: ', duplicate_apps[:10])

Number of duplicate apps:  1181


Examples of duplicate apps:  ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


Let's see if Instagram has duplicates

In [10]:
print(Google_header[0], '\t\t', Google_header[3])
for app in Google_data:
    name=app[0]
    if name == 'Instagram':
        print(app[0], '\t', app[3])

App 		 Reviews
Instagram 	 66577313
Instagram 	 66577446
Instagram 	 66577313
Instagram 	 66509917


To remove duplicate apps, we will only keep the most rated ones. To do so we will use a dictionary containing unique apps as keys and the highest number of reviews as values.

In [11]:
reviews_max={}

for app in Google_data:
    name=app[0]
    n_reviews=float(app[3])
    
    if name in reviews_max and reviews_max[name]<n_reviews:
        reviews_max[name]=n_reviews
    elif name not in reviews_max:
        reviews_max[name]=n_reviews
        

Let's see how many unique apps we have and check if we kept the highest entry for Instagram.

In [12]:
print('Unique apps: ', len(reviews_max))
print('Max reviews for Instagram: ', reviews_max['Instagram'])

Unique apps:  9659
Max reviews for Instagram:  66577446.0


The dictionary will be used to extract only unique apps with the highest number of ratings.
We have to iterate inside Google_data to compare each number of reviews with the dictionary.

When we find the app with the highest number or revies, we append it to `android_clean` and also to `already_added` because some apps may have duplicates with the highest number of ratings.

In [13]:
android_clean=[]
already_added=[]
for app in Google_data:
    name=app[0]
    n_reviews=float(app[3])
    if n_reviews==reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)

Let's check if we did it correctly with instagram:

In [14]:
print(Google_header[0], '\t\t', Google_header[3])
for app in android_clean:
    name=app[0]
    if name == 'Instagram':
        print(app[0], '\t', app[3])

App 		 Reviews
Instagram 	 66577446


The number of apps is the same as expected:

In [15]:
print(len(android_clean))

9659


## Removing non-english apps

Function to check if there are characters outside the ascii systea, so to check if some app name are not for english speakers

In [16]:
def check_ascii(a_string):
    for char in a_string:
        if ord(char)>127:
            return False
    return True
    

Let's try the funcion. Apparently it will exclude many apps just for containing one special character, for example 'Docs To Go™ Free Office Suite'. 

In [17]:
check_ascii('Docs To Go™ Free Office Suite')

False

So we will modify it to return  `False` when 3 non-ascii characters are counted

In [18]:
def check_ascii(a_string):
    non_ascii=0
    for char in a_string:
        if ord(char)>127:
            non_ascii+=1
    if non_ascii>3:
        return False
    else:
        return True

Let's check again:

In [19]:
print(check_ascii('Docs To Go™ Free Office Suite'))
print(check_ascii('Instachat 😜'))
print(check_ascii('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
True
False


Let's now use the function to clean our data sets

In [20]:
android_eng=[]

for app in android_clean:
    name=app[0]
    if check_ascii(name):
        android_eng.append(app)

explore_data(android_eng, 0, 3, True)
        

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


In [21]:
apple_eng=[]

for app in Apple_data:
    name=app[1]
    if check_ascii(name):
        apple_eng.append(app)

explore_data(apple_eng, 0, 3, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 6183
Number of columns: 16


## Isolating free apps
Android:

In [22]:
android_free=[]
for app in android_eng:
    price= app[7]
    if price == '0':
        android_free.append(app)
print(len(android_free))

8864


Apple:

In [23]:
apple_free=[]
for app in apple_eng:
    price= app[4]
    if price == '0.0':
        apple_free.append(app)
print(len(apple_free))

3222


## Most common free apps by genre

Our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play
2. If the app has a good response from users, we then develop it further.
3. If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set.

Let's create funcions to generate frequency tables and display them in descending order

In [24]:
def freq_table (dataset, index):
    frequency_table={}
    total=0
    for row in dataset:
        total+=1
        key=row[index]
        if key in frequency_table:
            frequency_table[key]+=1
        else:
            frequency_table[key]=1
            
    percent_table={}        
    for key in frequency_table:
        abs_value=frequency_table[key]
        percent_value=round(abs_value/total*100, 2)
        percent_table[key]=percent_value
    
    return percent_table
        

In [25]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [26]:
display_table(apple_free, -5) # prime genre

Games : 58.16
Entertainment : 7.88
Photo & Video : 4.97
Education : 3.66
Social Networking : 3.29
Shopping : 2.61
Utilities : 2.51
Sports : 2.14
Music : 2.05
Health & Fitness : 2.02
Productivity : 1.74
Lifestyle : 1.58
News : 1.33
Travel : 1.24
Finance : 1.12
Weather : 0.87
Food & Drink : 0.81
Reference : 0.56
Business : 0.53
Book : 0.43
Navigation : 0.19
Medical : 0.19
Catalogs : 0.12


In [27]:
display_table(android_free, 1) # Category

FAMILY : 18.91
GAME : 9.72
TOOLS : 8.46
BUSINESS : 4.59
LIFESTYLE : 3.9
PRODUCTIVITY : 3.89
FINANCE : 3.7
MEDICAL : 3.53
SPORTS : 3.4
PERSONALIZATION : 3.32
COMMUNICATION : 3.24
HEALTH_AND_FITNESS : 3.08
PHOTOGRAPHY : 2.94
NEWS_AND_MAGAZINES : 2.8
SOCIAL : 2.66
TRAVEL_AND_LOCAL : 2.34
SHOPPING : 2.25
BOOKS_AND_REFERENCE : 2.14
DATING : 1.86
VIDEO_PLAYERS : 1.79
MAPS_AND_NAVIGATION : 1.4
FOOD_AND_DRINK : 1.24
EDUCATION : 1.16
ENTERTAINMENT : 0.96
LIBRARIES_AND_DEMO : 0.94
AUTO_AND_VEHICLES : 0.93
HOUSE_AND_HOME : 0.82
WEATHER : 0.8
EVENTS : 0.71
PARENTING : 0.65
ART_AND_DESIGN : 0.64
COMICS : 0.62
BEAUTY : 0.6


In [28]:
display_table(android_free, 9) # Genres

Tools : 8.45
Entertainment : 6.07
Education : 5.35
Business : 4.59
Productivity : 3.89
Lifestyle : 3.89
Finance : 3.7
Medical : 3.53
Sports : 3.46
Personalization : 3.32
Communication : 3.24
Action : 3.1
Health & Fitness : 3.08
Photography : 2.94
News & Magazines : 2.8
Social : 2.66
Travel & Local : 2.32
Shopping : 2.25
Books & Reference : 2.14
Simulation : 2.04
Dating : 1.86
Arcade : 1.85
Video Players & Editors : 1.77
Casual : 1.76
Maps & Navigation : 1.4
Food & Drink : 1.24
Puzzle : 1.13
Racing : 0.99
Role Playing : 0.94
Libraries & Demo : 0.94
Auto & Vehicles : 0.93
Strategy : 0.91
House & Home : 0.82
Weather : 0.8
Events : 0.71
Adventure : 0.68
Comics : 0.61
Beauty : 0.6
Art & Design : 0.6
Parenting : 0.5
Card : 0.45
Casino : 0.43
Trivia : 0.42
Educational;Education : 0.39
Board : 0.38
Educational : 0.37
Education;Education : 0.34
Word : 0.26
Casual;Pretend Play : 0.24
Music : 0.2
Racing;Action & Adventure : 0.17
Puzzle;Brain Games : 0.17
Entertainment;Music & Video : 0.17
Casual;

To find out the most popular apps, we can calculate the average number of installs. For Google Play we have the column `Installs` and for Apple we have `rating_count_tot`

Apple store:

In [65]:
Apple_genres=freq_table(apple_free,-5)
for genre in Apple_genres:
    total=0
    len_genre=0
    for app in apple_free:
        genre_app=app[-5] # -5 is 'prime genre'
        if genre_app==genre:
            ratings=float(app[5]) #5 is rating count tot
            total+=ratings
            len_genre+=1
    average_ratings=round(total/len_genre)
    print(genre, ': ', round(average_ratings/1000,1), 'thousands')

Social Networking :  71.5 thousands
Photo & Video :  28.4 thousands
Games :  22.8 thousands
Music :  57.3 thousands
Reference :  74.9 thousands
Health & Fitness :  23.3 thousands
Weather :  52.3 thousands
Utilities :  18.7 thousands
Travel :  28.2 thousands
Shopping :  26.9 thousands
News :  21.2 thousands
Navigation :  86.1 thousands
Lifestyle :  16.5 thousands
Entertainment :  14.0 thousands
Food & Drink :  33.3 thousands
Sports :  23.0 thousands
Book :  39.8 thousands
Finance :  31.5 thousands
Education :  7.0 thousands
Productivity :  21.0 thousands
Business :  7.5 thousands
Catalogs :  4.0 thousands
Medical :  0.6 thousands


Most popular genres:
1. Navigation :  86.1 thousands
2. Reference :  74.9 thousands
3. Social Networking :  71.5 thousands
4. Music :  57.3 thousands
5. Weather :  52.3 thousands
6. Book :  39.8 thousands
7. Food & Drink :  33.3 thousands
8. Finance :  31.5 thousands
9. Travel :  28.2 thousands
10. Shopping :  26.9 thousands

Google:
we have the number of installs buy it's like this:

In [39]:
display_table(android_free, 5) #percentage apps by installs

1,000,000+ : 15.73
100,000+ : 11.55
10,000,000+ : 10.55
10,000+ : 10.2
1,000+ : 8.39
100+ : 6.92
5,000,000+ : 6.83
500,000+ : 5.56
50,000+ : 4.77
5,000+ : 4.51
10+ : 3.54
500+ : 3.25
50,000,000+ : 2.3
100,000,000+ : 2.13
50+ : 1.92
5+ : 0.79
1+ : 0.51
500,000,000+ : 0.27
1,000,000,000+ : 0.23
0+ : 0.05
0 : 0.01


We will use those ranges of installs as if they were a precise number. It's not ok but it's ok.
We have to clean the number of installs so to be compted as numbers

In [51]:
def clean_numbers(a_string):
    clean_string=''
    for char in a_string:
        if char!='+' and char!=',':
            clean_string+=char
    return clean_string

In [63]:
categories_android=freq_table(android_free, 1)

for category in categories_android:
    total=0
    len_category=0
    for app in android_free:
        category_app=app[1] # 1 is 'category'
        if category_app==category:
            n_installs=float(clean_numbers(app[5])) #5 is installs
            total+=n_installs
            len_category+=1
    average_installs=round(total/len_category)
    print(category, ': ', round(average_installs/1000000,1), 'millions')

ART_AND_DESIGN :  2.0 millions
AUTO_AND_VEHICLES :  0.6 millions
BEAUTY :  0.5 millions
BOOKS_AND_REFERENCE :  8.8 millions
BUSINESS :  1.7 millions
COMICS :  0.8 millions
COMMUNICATION :  38.5 millions
DATING :  0.9 millions
EDUCATION :  1.8 millions
ENTERTAINMENT :  11.6 millions
EVENTS :  0.3 millions
FINANCE :  1.4 millions
FOOD_AND_DRINK :  1.9 millions
HEALTH_AND_FITNESS :  4.2 millions
HOUSE_AND_HOME :  1.3 millions
LIBRARIES_AND_DEMO :  0.6 millions
LIFESTYLE :  1.4 millions
GAME :  15.6 millions
FAMILY :  3.7 millions
MEDICAL :  0.1 millions
SOCIAL :  23.3 millions
SHOPPING :  7.0 millions
PHOTOGRAPHY :  17.8 millions
SPORTS :  3.6 millions
TRAVEL_AND_LOCAL :  14.0 millions
TOOLS :  10.8 millions
PERSONALIZATION :  5.2 millions
PRODUCTIVITY :  16.8 millions
PARENTING :  0.5 millions
WEATHER :  5.1 millions
VIDEO_PLAYERS :  24.7 millions
NEWS_AND_MAGAZINES :  9.5 millions
MAPS_AND_NAVIGATION :  4.1 millions


Most popular categories:
1. COMMUNICATION :  38.5 millions
2. VIDEO_PLAYERS :  24.7 millions
3. SOCIAL :  23.3 millions
4. PHOTOGRAPHY :  17.8 millions
5. PRODUCTIVITY :  16.8 millions
6. GAME :  15.6 millions
7. ENTERTAINMENT :  11.6 millions
8. TOOLS :  10.8 millions
9. NEWS_AND_MAGAZINES :  9.5 millions
10. BOOKS_AND_REFERENCE :  8.8 millions


## Conclusions
Apart from social networking and navigation, which market is pretty saturated by giants, the most popular apps in both markets are in the genres/categories 
Book and reference